In [1]:
import utils.hgg_utils as hu
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.notebook import tqdm 
from model import unet
from utils.dice import dice_loss as dice
from utils.dice import dice_coef as dice_coef
from sklearn.utils import shuffle
from IPython import display
from tensorflow.keras.mixed_precision import experimental as mixed_precision
import time
import pickle

In [2]:
#policy = mixed_precision.Policy("float32") 
policy = mixed_precision.Policy("mixed_float16")
mixed_precision.set_policy(policy)

In [3]:
num_to_load = 5
n_slices = 155


# The val that varies between experiments

In [4]:
ds = 1

### Prepare to load in some input data and masks and remove outliers

In [5]:
patients = hu.get_each_normalized_hgg_folder()
patients = hu.remove_outliers(patients)

masks = hu.get_each_hgg_folder()
masks = hu.remove_outliers(masks)

patients, masks = shuffle(patients, masks, random_state=1)

### Train, Test indices

In [6]:
train_data_ratio = 0.8

In [7]:
train_start = 0
train_stop = int(np.round(train_data_ratio * len(patients)))
print(train_start)
train_stop

0


194

In [8]:
test_start = train_stop
test_stop = len(patients)
print(test_start)
test_stop

194


243

### Save paths for train 

In [9]:
train_data = patients[:test_start]
train_masks = masks[:test_start]

fname_train_data = "ds_"+str(ds)+"_train_data.pkl"
fname_train_masks = "ds_"+str(ds)+"_train_masks.pkl"

with open(fname_train_data, 'wb') as file_pi:
    pickle.dump(train_data, file_pi)
    
with open(fname_train_masks, 'wb') as file_pi:
    pickle.dump(train_masks, file_pi)

### Save the paths for testing 

In [10]:
test_data = patients[test_start:]
test_masks = masks[test_start:]

fname_test_data = "ds_"+str(ds)+"_test_data.pkl"
fname_test_masks = "ds_"+str(ds)+"_test_masks.pkl"

with open(fname_test_data, 'wb') as file_pi:
    pickle.dump(test_data, file_pi)
    
with open(fname_test_masks, 'wb') as file_pi:
    pickle.dump(test_masks, file_pi)

### Preallocate arrays to hold data & masks

In [11]:
some_data = np.ones([ num_to_load*155, 240, 240, 4])
some_masks = np.ones([ num_to_load*155, 240, 240, 1])

In [12]:
def load_n_brains(data, start, stop, paths, end):

    data_idx = 0
    num_slices = 155
    brains_seen = 0
    
    #for multimodal_tensor in tqdm(range(start, stop)):
    for multimodal_tensor in range(start, stop):

        if multimodal_tensor != end:
            four_channel_scan = hu.reshape_tensor_with_slices_first(
                                    hu.get_a_multimodal_tensor( 
                                                paths[multimodal_tensor] 
                                    )[data_idx]
            )
            #print(paths[multimodal_tensor])


            for slic in range(num_slices):
                data[slic+(num_slices*brains_seen),:,:,:] = four_channel_scan[slic,:,:,:]

            brains_seen += 1
        else:
            break
        
    #print(multimodal_tensor)
    return data

In [13]:
def load_n_masks(data, start, stop, paths, end):

    data_idx = 0
    num_slices = 155
    brains_seen = 0

    for mask_idx in range(start, stop):
        if mask_idx != end:

            mask =  hu.reshape_tensor_with_slices_first(
                                    hu.convert_mask_to_binary_mask(
                                         hu.get_a_mask_tensor( paths[mask_idx] )

                                   )
            )
            #print(paths[mask_idx])


            for slic in range(num_slices):
                data[slic+(num_slices*brains_seen),:,:,:] = mask[slic,:,:,:]

            brains_seen += 1
        else:
            break
            
    return data

### Train

In [14]:
chunks = 39

beg = 0
end = num_to_load*155
truncated = 155*(train_stop - ( (chunks-1) * num_to_load  ) )

#truncated
#155*(train_stop - ( (chunks-1) * num_to_load  ) )

my_opt = tf.keras.optimizers.Adam(learning_rate=1e-4)

In [15]:

start_time = time.time()

for run in tqdm(range(5)):
    
    print("********************")
    print("Run:", run)
    
    model = unet( input_size=(240,240,4), ds=ds )
    # Save architecture
    model_json_name = "unet_ds_{}.json".format(ds)
    with open(model_json_name, "w") as json_file:
        json_file.write(model.to_json())
        
    model.compile(optimizer=my_opt, loss=dice, metrics=[dice_coef])
    
    run_history = []

    for epoch in tqdm(range(20)):
        
        epoch_history = []
        
        for i in range(chunks):
            print("Loading chunk of data...")
            some_data = load_n_brains(some_data,  (num_to_load*i), (num_to_load*i)+num_to_load, patients, train_stop).astype(np.float32)
            some_masks = load_n_masks(some_masks, (num_to_load*i), (num_to_load*i)+num_to_load, masks, train_stop).astype(np.float32)

            some_data, some_masks = shuffle(some_data, some_masks, random_state=1)

            if num_to_load*i+num_to_load <= train_stop:
                history = model.fit(some_data[beg:end,...], some_masks[beg:end,...], validation_split=0.2, epochs=1, batch_size=32)

            else:
                history = model.fit(some_data[beg:truncated,...], some_masks[beg:truncated,...], validation_split=0.2, epochs=1, batch_size=32)


            epoch_history.append(history.history)
        print("Epoch", epoch, "completed")
        print("Elapsed time:", (time.time() - start_time)/60.0, "minutes" )
        run_history.append(epoch_history)
    
    print()
    print("Saving run", run, "loss etc.")
    history_name = "ds_"+str(ds)+"_run_" + str(run) +"_histories.pkl"

    with open(history_name, 'wb') as file_pi:
        pickle.dump(run_history, file_pi)
    
    model_weights_name = "ds_"+str(ds)+"_run_" + str(run) +"_model_weights.h5"
    
    print("Saving run", run, "model weights as", model_weights_name)
    model.save_weights(model_weights_name)
    
    del model
    
print("Total time:", (time.time() - start_time)/60.0, "minutes"  )

********************
Run: 0


Loading chunk of data...
Train on 1240 samples, validate on 310 samples
1240/1240 [==============================] - 45s 36ms/sample - loss: 0.9648 - dice_coef: 0.0352 - val_loss: 0.9640 - val_dice_coef: 0.0358
Loading chunk of data...
Train on 1240 samples, validate on 310 samples
1240/1240 [==============================] - 28s 23ms/sample - loss: 0.9603 - dice_coef: 0.0398 - val_loss: 0.9570 - val_dice_coef: 0.0432
Loading chunk of data...
Train on 1240 samples, validate on 310 samples
1240/1240 [==============================] - 28s 23ms/sample - loss: 0.9666 - dice_coef: 0.0335 - val_loss: 0.9675 - val_dice_coef: 0.0328
Loading chunk of data...
Train on 1240 samples, validate on 310 samples
1240/1240 [==============================] - 28s 23ms/sample - loss: 0.9506 - dice_coef: 0.0494 - val_loss: 0.9566 - val_dice_coef: 0.0428
Loading chunk of data...
Train on 1240 samples, validate on 310 samples
1240/1240 [==============================] - 28s 23ms/sample - loss: 0.9652 - dice_c

1240/1240 [==============================] - 25s 20ms/sample - loss: 0.9723 - dice_coef: 0.0277 - val_loss: 0.9649 - val_dice_coef: 0.0354
Loading chunk of data...
Train on 496 samples, validate on 124 samples
496/496 [==============================] - 10s 21ms/sample - loss: 0.9733 - dice_coef: 0.0273 - val_loss: 0.9765 - val_dice_coef: 0.0237
Epoch 1 completed
Elapsed time: 24.5878977338473 minutes
Loading chunk of data...
Train on 1240 samples, validate on 310 samples
1240/1240 [==============================] - 25s 20ms/sample - loss: 0.9482 - dice_coef: 0.0519 - val_loss: 0.9564 - val_dice_coef: 0.0433
Loading chunk of data...
Train on 1240 samples, validate on 310 samples
1240/1240 [==============================] - 25s 20ms/sample - loss: 0.9519 - dice_coef: 0.0480 - val_loss: 0.9510 - val_dice_coef: 0.0492
Loading chunk of data...
Train on 1240 samples, validate on 310 samples
1240/1240 [==============================] - 27s 21ms/sample - loss: 0.9613 - dice_coef: 0.0386 - val_

1240/1240 [==============================] - 25s 20ms/sample - loss: 0.9536 - dice_coef: 0.0464 - val_loss: 0.9502 - val_dice_coef: 0.0498
Loading chunk of data...
Train on 1240 samples, validate on 310 samples
1240/1240 [==============================] - 25s 20ms/sample - loss: 0.9545 - dice_coef: 0.0455 - val_loss: 0.9579 - val_dice_coef: 0.0418
Loading chunk of data...
Train on 1240 samples, validate on 310 samples
1240/1240 [==============================] - 25s 20ms/sample - loss: 0.9698 - dice_coef: 0.0301 - val_loss: 0.9617 - val_dice_coef: 0.0386
Loading chunk of data...
Train on 496 samples, validate on 124 samples
496/496 [==============================] - 10s 21ms/sample - loss: 0.9707 - dice_coef: 0.0298 - val_loss: 0.9743 - val_dice_coef: 0.0260
Epoch 3 completed
Elapsed time: 47.55670081774394 minutes
Loading chunk of data...
Train on 1240 samples, validate on 310 samples
1240/1240 [==============================] - 25s 20ms/sample - loss: 0.9433 - dice_coef: 0.0568 - val

1240/1240 [==============================] - 26s 21ms/sample - loss: 0.9406 - dice_coef: 0.0592 - val_loss: 0.9427 - val_dice_coef: 0.0570
Loading chunk of data...
Train on 1240 samples, validate on 310 samples
1240/1240 [==============================] - 25s 20ms/sample - loss: 0.9472 - dice_coef: 0.0529 - val_loss: 0.9460 - val_dice_coef: 0.0530
Loading chunk of data...
Train on 1240 samples, validate on 310 samples
1240/1240 [==============================] - 25s 20ms/sample - loss: 0.9490 - dice_coef: 0.0510 - val_loss: 0.9455 - val_dice_coef: 0.0546
Loading chunk of data...
Train on 1240 samples, validate on 310 samples
1240/1240 [==============================] - 25s 20ms/sample - loss: 0.9504 - dice_coef: 0.0495 - val_loss: 0.9540 - val_dice_coef: 0.0457
Loading chunk of data...
Train on 1240 samples, validate on 310 samples
1240/1240 [==============================] - 26s 21ms/sample - loss: 0.9670 - dice_coef: 0.0332 - val_loss: 0.9581 - val_dice_coef: 0.0422
Loading chunk of 

1240/1240 [==============================] - 25s 20ms/sample - loss: 0.9492 - dice_coef: 0.0506 - val_loss: 0.9475 - val_dice_coef: 0.0527
Loading chunk of data...
Train on 1240 samples, validate on 310 samples
1240/1240 [==============================] - 25s 20ms/sample - loss: 0.9509 - dice_coef: 0.0489 - val_loss: 0.9495 - val_dice_coef: 0.0503
Loading chunk of data...
Train on 1240 samples, validate on 310 samples
1240/1240 [==============================] - 27s 22ms/sample - loss: 0.9345 - dice_coef: 0.0656 - val_loss: 0.9371 - val_dice_coef: 0.0625
Loading chunk of data...
Train on 1240 samples, validate on 310 samples
1240/1240 [==============================] - 25s 20ms/sample - loss: 0.9420 - dice_coef: 0.0579 - val_loss: 0.9408 - val_dice_coef: 0.0581
Loading chunk of data...
Train on 1240 samples, validate on 310 samples
1240/1240 [==============================] - 25s 20ms/sample - loss: 0.9440 - dice_coef: 0.0559 - val_loss: 0.9399 - val_dice_coef: 0.0602
Loading chunk of 

MemoryError: Unable to allocate 1.33 GiB for an array with shape (1550, 240, 240, 4) and data type float32

In [ ]:
#print( run_history[0].history )